In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import _pickle as pkl
import pandas as pd
import os,io,sys,glob, h5py, csv, time
import matplotlib.pyplot as plt
import json

from dlab import generalephys as ephys
from dlab import generalephys_mua as ephys_mua
#from neuropixels import cell_pairs as pairs

#from neuropixels import utils_pipeline as utils
#from neuropixels import analysis_pipeline as analysis

from dlab import analysis_pipeline as analysis
from dlab.continuous_traces import gaussian_filter1d
from scipy.signal import find_peaks
from scipy.stats import pearsonr, spearmanr, zscore
from itertools import combinations 
#from neuropixels import sorting_quality_editing as sq
from sklearn.manifold import TSNE
from scipy.cluster.vq import kmeans2
#from dlab import depth_estimation as depth
#from dlab import PCA as pca
from scipy import stats

import seaborn as sns
sns.set_style('white')
%matplotlib inline
%load_ext autoreload
%autoreload 2
sns.set_style("white")
import warnings
warnings.filterwarnings('ignore')

In [1]:
def psth(df,df_reaches,start,end,binsize):

    tensor = []
    hists = []
    each_unit = []
    behavior = []
    units = []
    reach = []
    all_reach = []
    all_rastor = []
    peakt = []

    for i,times in enumerate(df.times): #compare that unit's spike times to each reach max
        t = np.array(times) #for reach unit create an array of that unit's spike times
        hists = []
        for j,tmax in enumerate(df_reaches.rMax_t): #for each unit 
            a = tmax+start
            b = tmax+end
            rd = np.array(t[(t >= a) & (t <= b)]) #find if that unit spiked within designated timeframes around reachmax
            edges=np.arange(a,b,binsize) #designated bins around this iteration of reachmax
            hist=np.histogram(rd,edges)[0] #bin spikes into timeframe
            
            #maxfr = np.max(hist)
            #print(maxfr)
            #maxfr_t = np.where(edges==maxfr)
            #print(maxfr_t)
            #hist=list(hist)
            #hist.extend([2])
            hists.append(hist) # kim changed to *0.5 5.23.23
            peakt.append(df.peakup[i])

        mn = np.mean(hists,axis=0)
        all_rastor.append(hists)
        all_reach.append(mn)
    return all_reach,all_rastor,peakt

# imports

In [ ]:
df = pd.read_json(r'')

In [ ]:
# use dataframe from after behavior alignment

In [ ]:
df_run = df[df.mod_up=='yes'][df.epochFR>5].sort_values(by=['peakup'])

In [ ]:
avereach,raster,peaks = psth(df_run,dfr,-.5,.5,0.001)

In [ ]:
rast_maps = []
for i,rast in enumerate(raster):
        rast_maps.extend(rast)

In [ ]:
# this gets rid of double counted units (if a bin has integer 2 instead of 1, just cleans up the figure)
new_rastmap = []
for i in rast_maps:
    new_rast = []
    for j in i:
        if j > 0:
            new_rast.append(1)
        if j == 0:
            new_rast.append(0)
    new_rastmap.append(new_rast)

In [ ]:
clust = []
for i in np.arange(1,np.shape(dfr)[0]+1,1):
    clust.extend(np.ones(np.shape(df_run)[0])*i)

In [ ]:
df_clust = pd.DataFrame(data=clust,columns=['clust'])

In [ ]:
df_clust['rast'] = new_rastmap

In [ ]:
colors = ['mediumpurple','cornflowerblue','violet','turquoise','mediumpurple','lightgreen','cornflowerblue','violet','turquoise','mediumpurple','lightgreen','cornflowerblue','violet','turquoise','mediumpurple','lightgreen','cornflowerblue','violet','turquoise','mediumpurple','lightgreen','cornflowerblue','mediumpurple','lightgreen','cornflowerblue','violet','turquoise','mediumpurple','lightgreen','cornflowerblue']
#colors = ['mediumpurple','salmon','cornflowerblue','lightgreen','turquoise','mediumpurple','salmon','cornflowerblue','lightgreen','turquoise','mediumpurple','salmon','cornflowerblue','lightgreen','turquoise','mediumpurple','salmon','cornflowerblue','lightgreen','turquoise','mediumpurple','salmon']
plt.figure()
plt.subplots(figsize=(8,4))
axes=plt.gca()
linewidth = 0.5
markersize=3
for i,rast in enumerate(df_clust.rast):
    cl = int(df_clust.clust[i])
    color = colors[cl]
    x = np.arange(-0.5,0.5,0.001)
    y = np.array(rast)
    axes.plot(x[:-1], y*(i+2),"o",markersize=markersize,mew=linewidth,color=color)
    axes.set_xlabel(r'$time \/ (s)$',fontsize=20)
    #axes.set_ylabel(r'$trials$',fontsize=20)
    axes.tick_params(axis='both',labelsize=20)
    #axes.axvline(0,linestyle='--',color='grey')
    axes.spines['top'].set_visible(False);axes.yaxis.set_ticks_position('left')
    axes.spines['right'].set_visible(False);axes.xaxis.set_ticks_position('bottom')
#plt.savefig(r'C:\Users\Kim\Documents\conferences\ARMartin\testfog2.svg')
plt.ylim(np.shape(rast_maps)[0],3)
#plt.savefig(r'Y:\BIOElectricsLab\Kim\ConferenceGrantFigs\MyelinGrant\cntrl_raster\komiyamafig.png')
plt.show()